## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Abe Mankavil - Group 22

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [392]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [393]:
df_X, s_y = load_diabetes(return_X_y=True, as_frame=True)
df_X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


## Q1.2 Define a function that creates a linear least squares regression model and a function to predict a continuous value given a linear regression model
The first function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

The second function should take in two parameters, `beta_hat` - the model generated from the `get_linear_regression` function, and `df_X` - that has the attribute values at which we want to predict a continuous value.  We assume that the format and ordering of `df_X` used to create the model and `df_X` used to generate predictions are consistent. 

In [394]:
def get_linear_regression_model(df_X, s_y):
    ones = pd.Series(np.ones(len(df_X)))
    beta_hat, _, _, _ = np.linalg.lstsq(pd.concat([pd.DataFrame({'intercept':ones}),df_X], axis=1),s_y,rcond=None)
    return beta_hat


In [395]:
# code to check beta_hat
np.random.seed(23)
beta_hat = get_linear_regression_model( pd.DataFrame(np.random.random((34,4))), pd.Series(np.random.random(34)*10.0) )
beta_hat

array([ 4.18818425,  1.77890808,  0.74032569, -1.3506416 ,  0.14535984])

In [396]:
def predict_linear_regression_value(beta_hat, df_X):
    ones = pd.Series(np.ones(len(df_X)))
    y_hat = np.matmul(pd.concat([ones,df_X], axis=1),beta_hat)
    return y_hat

In [397]:
predicted_vals = predict_linear_regression_value(beta_hat, pd.DataFrame(np.random.random((3,4))))
predicted_vals


0    4.036695
1    4.395026
2    3.908847
dtype: float64

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [398]:
def partition_data( df_X, s_y, k ):
    dict_k_df_X, dict_k_s_y = {}, {}
    n = len(df_X)
    for i in range(1,k+1):
        X_train = df_X[n*(i-1)//k:n*i//k]
        dict_k_df_X[i] = X_train
        y_train = s_y[n*(i-1)//k:n*i//k]
        dict_k_s_y[i] = y_train
    return (dict_k_df_X, dict_k_s_y)
        



In [399]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )


In [400]:
# Check fold sizes
total = 0
for i in range(1,len(dict_k_df_X)+1):
    total += len(dict_k_df_X[i])
    print(f"Fold {i} length of dataframe is {len(dict_k_df_X[i])} and length of series is {len(dict_k_s_y[i])} ")
print(f"The sum of the number of elements in each fold is {total} and there are {len(df_X)} rows in the original df")

Fold 1 length of dataframe is 88 and length of series is 88 
Fold 2 length of dataframe is 88 and length of series is 88 
Fold 3 length of dataframe is 89 and length of series is 89 
Fold 4 length of dataframe is 88 and length of series is 88 
Fold 5 length of dataframe is 89 and length of series is 89 
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [401]:
def get_mae(s_y, s_y_hat):
    mae = 0
    for i in range(len(s_y)):
        mae += np.abs((s_y[i] - s_y_hat[i]))/len(s_y)
    return mae

In [402]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

You must use your helper functions that you wrote above, `get_linear_regression_model`, `predict_linear_regression_value` and `get_mae`. 

In [403]:
mae = np.array([])
for k in dict_k_df_X.keys():
    not_k = [i for i in dict_k_df_X.keys() if i != k]
    
    df_X_train = pd.concat([dict_k_df_X[i] for i in not_k], axis=0).reset_index(drop=True)
    s_y_train = pd.concat([dict_k_s_y[i] for i in not_k], axis=0).reset_index(drop=True)


    df_X_test = dict_k_df_X[k].reset_index(drop=True)
    s_y_test = dict_k_s_y[k].reset_index(drop=True)

    beta_hat = get_linear_regression_model(df_X_train, s_y_train)
    s_y_hat = predict_linear_regression_value(beta_hat,df_X_test)
    mae = np.append(mae, get_mae(s_y_test,s_y_hat))  



In [404]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))

The min MAE is 42.55, the max MAE is 47.93, and the mean MAE is 44.28


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [412]:
df_X, s_y = load_iris(return_X_y=True, as_frame=True)


## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [406]:
dict_k_df_X, dict_k_s_y = partition_data(df_X,s_y,5)
total = 0
for i in range(1,len(dict_k_df_X)+1):
    total += len(dict_k_df_X[i])
    print(f"Fold {i} length of dataframe is {len(dict_k_df_X[i])} and length of series is {len(dict_k_s_y[i])} ")
print(f"The sum of the number of elements in each fold is {total} and there are {len(df_X)} rows in the original df")

Fold 1 length of dataframe is 30 and length of series is 30 
Fold 2 length of dataframe is 30 and length of series is 30 
Fold 3 length of dataframe is 30 and length of series is 30 
Fold 4 length of dataframe is 30 and length of series is 30 
Fold 5 length of dataframe is 30 and length of series is 30 
The sum of the number of elements in each fold is 150 and there are 150 rows in the original df


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [407]:
def get_acc(s_1, s_2):
    return np.sum(np.equal(s_1,s_2))/len(s_1)

In [408]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [409]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
outer_acc = np.array([])
for k in dict_k_df_X.keys():
    not_k = [i for i in dict_k_df_X.keys() if i != k]
    
    df_X_train = pd.concat([dict_k_df_X[i] for i in not_k], axis=0).reset_index(drop=True)
    s_y_train = pd.concat([dict_k_s_y[i] for i in not_k], axis=0).reset_index(drop=True)

    df_X_test = dict_k_df_X[k].reset_index(drop=True)
    s_y_test = dict_k_s_y[k].reset_index(drop=True)

    dict_mean_acc = {}
    for pos_min_impurity in sorted(possible_min_impurity_decrease):
        
        print(f"Testing {pos_min_impurity} min impurity decrease")
        inner_acc = np.array([])
        dict_k_df_X_inner, dict_k_s_y_inner = partition_data(df_X_train,s_y_train,4)
        
        for j in dict_k_df_X_inner.keys():
            not_k = [i for i in dict_k_df_X_inner.keys() if i != j]
            
            df_X_train_inner = pd.concat([dict_k_df_X_inner[i] for i in not_k], axis=0).reset_index(drop=True)
            s_y_train_inner = pd.concat([dict_k_s_y_inner[i] for i in not_k], axis=0).reset_index(drop=True)

            df_X_test_inner = dict_k_df_X_inner[j].reset_index(drop=True)
            s_y_test_inner = dict_k_s_y_inner[j].reset_index(drop=True)
            clf = tree.DecisionTreeClassifier(criterion="gini",min_impurity_decrease=pos_min_impurity)
            clf = clf.fit(df_X_train_inner, s_y_train_inner)
            pred = np.array(clf.predict(df_X_test_inner))
            inner_acc = np.append(inner_acc,get_acc(s_y_test_inner,pred))
        print(f"Average accuracy over {j} folds is {inner_acc.mean()}")

        if inner_acc.mean() not in dict_mean_acc:
            dict_mean_acc[inner_acc.mean()] = pos_min_impurity

    print(f"Best min impurity decrease is {dict_mean_acc[max(dict_mean_acc.keys())]}\n")
        
    # Use best min impurity decrease to train model
    clf = tree.DecisionTreeClassifier(criterion="gini",min_impurity_decrease=0.1)
    clf = clf.fit(df_X_train, s_y_train)
    pred = np.array(clf.predict(df_X_test))

    # outer accuracy calculation 
    this_acc = get_acc(s_y_test,pred)
    outer_acc = np.append(outer_acc,this_acc) # make sure and calculate this_acc in your loop

Testing 0.1 min impurity decrease
Average accuracy over 4 folds is 0.7333333333333334
Testing 0.25 min impurity decrease
Average accuracy over 4 folds is 0.525
Testing 0.3 min impurity decrease
Average accuracy over 4 folds is 0.16666666666666666
Testing 0.4 min impurity decrease
Average accuracy over 4 folds is 0.08333333333333333
Best min impurity decrease is 0.1

Testing 0.1 min impurity decrease
Average accuracy over 4 folds is 0.6
Testing 0.25 min impurity decrease
Average accuracy over 4 folds is 0.23333333333333334
Testing 0.3 min impurity decrease
Average accuracy over 4 folds is 0.08333333333333333
Testing 0.4 min impurity decrease
Average accuracy over 4 folds is 0.0
Best min impurity decrease is 0.1

Testing 0.1 min impurity decrease
Average accuracy over 4 folds is 0.9166666666666666
Testing 0.25 min impurity decrease
Average accuracy over 4 folds is 0.5833333333333333
Testing 0.3 min impurity decrease
Average accuracy over 4 folds is 0.5833333333333333
Testing 0.4 min impu

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [414]:
print(outer_acc)
print(f"Min: {outer_acc.min()}; Max: {outer_acc.max()}; Mean: {outer_acc.mean()}")

[1.         0.96666667 0.83333333 0.93333333 0.83333333]
Min: 0.8333333333333334; Max: 1.0; Mean: 0.9133333333333333
